In [1]:
import numpy as np


🛰️ ORBITAL MATHEMATICS ACADEMY
MISSION: SATELLITE ATTITUDE OPTIMIZATION
STATUS: LIVE OPERATIONS

🎯 MISSION SCENARIO: EARTHWATCH-1 SATELLITE
You are the flight software engineer for EarthWatch-1, an Earth observation satellite currently orbiting at 700km altitude. The satellite has three critical systems that must work simultaneously:
🔭 Primary Systems:

High-Resolution Camera: Must point at Sydney, Australia for disaster monitoring
Solar Array: Must track the Sun for power generation
Communication Antenna: Must maintain link with ground station

⚠️ THE CHALLENGE:
All three systems cannot be optimally oriented simultaneously. You need to find the best compromise attitude that satisfies all constraints.
🧮 MATHEMATICAL MISSION PARAMETERS

In [ ]:
# create class to group rotation functions


class RotationMatrices:
    
    """this classs groups my linear transformation functions"""

    @staticmethod
    def safe_cos(ang_deg):
      ang = np.radians(ang_deg)
      result = np.cos(ang)

      if abs(result) < 1e-15:
        return 0.0
      else:
        return result
       

    @staticmethod
    def rotation_matrix_x(ang_deg):
   
     """This rotates on the x-axis, which means x stays still and y and z rotate around it"""

     ang = np.radians(ang_deg)

     Rx = np.array([
        [1,0,0],
        [0, RotationMatrices.safe_cos(ang_deg), -np.sin(ang)],
        [0, np.sin(ang), RotationMatrices.safe_cos(ang_deg)]
        ])
     
     return Rx

    @staticmethod
    def rotation_matrix_y(ang_deg):
   
     """This rotates on the y-axis, which means y stays still and x and z rotate around it"""

     ang = np.radians(ang_deg)

     Ry = np.array([
        [RotationMatrices.safe_cos(ang_deg),0,np.sin(ang)],
        [0, 1, 0],
        [-np.sin(ang), 0 , RotationMatrices.safe_cos(ang_deg)]
        ])
     
     return Ry
    
    @staticmethod
    def rotation_matrix_z(ang_deg):
   
     """This rotates on the z-axis, which means z stays still and y and x rotate around it"""

     ang = np.radians(ang_deg)

     Rz = np.array([
        [RotationMatrices.safe_cos(ang_deg),-np.sin(ang),0],
        [np.sin(ang), RotationMatrices.safe_cos(ang_deg), 0],
        [0, 0 , 1]
        ])
     
     return Rz
    
    @staticmethod
    def euler_angles_to_rotation_matrix(roll_deg, pitch_deg, yaw_deg, sequence='ZYX'):
      
      """This gives the final rotation matrix given tthe roll, pitch and yaw degree and the desired rotation sequence"""

      roll = np.radians(roll_deg)
      pitch = np.radians(pitch_deg)
      yaw = np.radians(yaw_deg) 

      Rx = RotationMatrices.rotation_matrix_x(roll_deg)
      Ry = RotationMatrices.rotation_matrix_y(pitch_deg)
      Rz = RotationMatrices.rotation_matrix_z(yaw_deg)

      if sequence == 'ZYX':
        return Rx @ Ry @ Rz
      elif sequence == 'XYZ':
        return Rz @ Ry @ Rx
      else:
        raise ValueError(f"Sequence {sequence} not implemented")
      
    @staticmethod
    def rotation_matrix_eci_to_ecef(greenwich_hour_angle_deg):
      """Earth rotates around the z axis so this is simply a z-axis at the greenwich hour angle"""
      
      R_eci_to_ecef = RotationMatrices.rotation_matrix_z(greenwich_hour_angle_deg)
      return R_eci_to_ecef
    

      
      

class SpacecraftState:

  """This is my spaceship"""
  def __init__(self, position, velocity, earth_rotation_rate = 15.04):
    """this is where is is and where it is going"""
    self.position = np.array(position)
    self.velocity = np.array(velocity)
    self.earth_rotation_rate = earth_rotation_rate
  

  def rotation(self, rotation_matrix):
    """This is how we manouvre my spaceship in space"""
    self.position = rotation_matrix @ self.position
    self.velocity = rotation_matrix @ self.velocity

  def eci_to_ecef(self, time_hours, eci_position):

    """Apply the rotationn matrix to convert from eci to ecef.
    To do this we need to calculate the greenwich hour angle. We need the time since 
    the reference epoc and the rate of rotation in degrees per hour"""

    gha = self.earth_rotation_rate*time_hours
    ecef_position = np.array(eci_position)

    Reci_to_ecef = RotationMatrices.rotation_matrix_eci_to_ecef(gha)

    ecef_position = Reci_to_ecef @ eci_position

    return ecef_position

def geographic_to_ecef(lat_deg, lon_deg, alt_km, earth_radius=6371):
    lat_rad = np.radians(lat_deg)
    lon_rad = np.radians(lon_deg)
    
    # ECEF coordinates
    x = (earth_radius + alt_km) * cos(lat_rad) * cos(lon_rad)
    y = (earth_radius + alt_km) * cos(lat_rad) * sin(lon_rad)  
    z = (earth_radius + alt_km) * sin(lat_rad)
    
    return [x, y, z]

In [ ]:

# Satellite position in ECI coordinates (km)

satellite_pos = [7000, 0, 0]  # 700km altitude, over equator

# Target: Sydney, Australia  
target_lat = -33.8688  # degrees
target_lon = 151.2093  # degrees

# Sun direction in ECI (simplified)
sun_direction = [0.707, 0.707, 0]  # 45° from X-axis in XY plane

# Ground station direction (straight down to Earth center)
earth_direction = [-1, 0, 0]  # Opposite to satellite position